# リポジトリクローン

In [1]:
!git clone https://github.com/tengyu1998/SCI

Cloning into 'SCI'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 40 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [2]:
%cd SCI

/content/SCI


# モデルロード

In [3]:
import torch
from model_test import Network

model = Network()
model_dict = torch.load('model/demo.pt', map_location='cpu')
model.load_state_dict(model_dict)

model.eval()

Network(
  (enhance): EnhanceNetwork(
    (out_conv): Sequential(
      (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): Sigmoid()
    )
  )
)

# ONNX変換

In [4]:
def convert_to_onnx(net, file_name='output.onnx', input_shape=(512, 512), device='cpu'):
    input = torch.randn(1, 3, input_shape[0], input_shape[1]).to(device)

    input_layer_names = ['input_image']
    output_layer_names = ['output_image']

    torch.onnx.export(
        net, 
        input, 
        file_name, 
        verbose=True,
        opset_version=13,
        input_names=input_layer_names,
        output_names=output_layer_names,
    )

In [5]:
convert_to_onnx(model, input_shape=(512, 512))

graph(%input_image : Float(1, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=0, device=cpu),
      %enhance.out_conv.0.weight : Float(3, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %enhance.out_conv.0.bias : Float(3, strides=[1], requires_grad=1, device=cpu),
      %enhance.out_conv.2.weight : Float(3, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %enhance.out_conv.2.bias : Float(3, strides=[1], requires_grad=1, device=cpu),
      %enhance.out_conv.4.weight : Float(3, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %enhance.out_conv.4.bias : Float(3, strides=[1], requires_grad=1, device=cpu),
      %25 : Float(requires_grad=0, device=cpu),
      %26 : Float(requires_grad=0, device=cpu),
      %27 : Float(requires_grad=0, device=cpu),
      %28 : Float(requires_grad=0, device=cpu)):
  %7 : Float(1, 3, 512, 512, strides=[786432, 262144, 512, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], grou